# Run the NetLogo`HeatDiffusion` model through pyNetLogo

`HeatDiffusion` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. View the [source code](https://github.com/csdms/bmi-example-pynetlogo/blob/main/heat/HeatDiffusion.nlogo) for the model on GitHub.

[NetLogo](https://ccl.northwestern.edu/netlogo/) is required to run this notebook.
Once you've [downloaded and installed](https://ccl.northwestern.edu/netlogo/bind/article/getting-started-with-netlogo.html) NetLogo,
set the path to the install location in the `NETLOGO_HOME` variable below.
The current value of `NETLOGO_HOME` is where NetLogo is installed on the [EarthscapeHub](https://csdms.colorado.edu/wiki/JupyterHub) *lab* and *jupyter* Hubs.

In [ ]:
NETLOGO_HOME = "/opt/netlogo-6.1.1/"

Import the Python libraries we'll use below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pynetlogo

Start NetLogo in headless mode, using the `NETLOGO_HOME` variable.

In [ ]:
netlogo = pynetlogo.NetLogoLink(netlogo_home=NETLOGO_HOME, gui=False)

Load the `HeatDiffusion` model.
Call the *setup* command to initialize the model, using default values for all parameters.

In [ ]:
netlogo.load_model("../heat/HeatDiffusion.nlogo")
netlogo.command("setup")

The `temperature` variable is the primary output of the `HeatDiffusion` model.
It's defined on the [patches](https://ccl.northwestern.edu/netlogo/bind/primitive/patches.html) of the model.
Patches in NetLogo are analogous to grid cells in a finite difference model.

Get the initial temperature distribution on the plate.

In [ ]:
temperature = netlogo.patch_report("temperature")

The plate temperature values are returned as a *pandas* `DataFrame`.

In [ ]:
type(temperature)

Show the values of the `temperature` variable.

In [ ]:
temperature.values

Try a naive visualization of the temperature values.

In [ ]:
plt.imshow(temperature.values)

What are we seeing here?
Get some more information from the `HeatDiffusion` model.

Get the temperature values at the edges of the plate. These values are held constant with time.

In [ ]:
temp_top = netlogo.report('top-temp')
temp_bottom = netlogo.report('bottom-temp')
temp_left = netlogo.report('left-temp')
temp_right = netlogo.report('right-temp')
edge_temp = [temp_top, temp_right, temp_bottom, temp_left]
edge_temp

What is the size of the plate?

In [ ]:
plate_size = netlogo.report("plate-size")
plate_size

Given this information,
and in studying the NetLogo code in `HeatDiffusion`,
we can interpret the plot above.

The model domain of the plate is centered at `[25,25]` in the plot,
and extends 15 patches in each direction,
so the plate--the model domain--extends from 10 to 40 in each direction.
The edges, which buffer the plate temperature,
extend an additional 10 patches in each direction.
The warmest temperature is on the right, in yellow,
and the coolest temperature is on the left, in blue.
The corners are set to the mean of the two intersecting edge temperatures.
The legend is stored on the left, outside of the model domain.

So, a better plot is the 30 x 30 interior of the `temperature` variable.
Import a helper function, *plot_temperature*, to do this.
The *plot_temperature* function takes the temperature values as a NumPy array,
as well as the plate size, which equates to the model domain.
In the plot,
I included one patch outside the model domain to show the initial edge temperatures.

In [ ]:
from helpers import plot_temperature

plot_temperature(temperature.values, plate_size)

What value of thermal diffusivity is used by the model?
From examination of the `HeatDiffusion` source,
the units are $cm^2 s^{-1}$.

In [ ]:
netlogo.report("heat-diffusivity")

What is the initial time in the model?

In [ ]:
dt = 0.1  # seconds, from model source
netlogo.report("ticks") * dt

Run the model for 200 time steps (20 seconds of model time).

In [ ]:
netlogo.command('repeat 200 [go]')

What is the current model time?

In [ ]:
netlogo.report("ticks") * dt

How has the temperature field has evolved?

In [ ]:
new_temperature = netlogo.patch_report("temperature")
plot_temperature(new_temperature.values, plate_size)

Diffusion!

Shut down the link to NetLogo when we're finished.

In [ ]:
netlogo.kill_workspace()